### 감정분석
* 텍스트에 숨겨져있는 저자의 의도/정보를 찾아내는 방법
* Opinion Mining, Sentiment Mining
* 긍정/부정으로 나눠지는 형태
* text classification

### 텍스트 분류
* 벡터형태로 된 텍스트를 분류
* 주로 딥러닝(LSTM) 사용
* 예) QnA, 챗봇(답변), 질문분류,...

In [1]:
# 파일 읽어오는 함수
def readFile(path):
    with open(path, encoding='utf-8') as f:
        doc = []
        for line in f.read().splitlines()[1:]:
            doc.append(line.split('\t')[1:])
    return doc

In [3]:
# NSMC 학습데이터와 테스트데이터 파일을 읽어오기
train = readFile('ratings_train.txt')
test = readFile('ratings_test.txt')

In [4]:
train[:3]

[['아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['너무재밓었다그래서보는것을추천한다', '0']]

In [5]:
# 전처리 함수
def preprocessing(txt):
    import re
    txt = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', txt)
    sw = set()
    with open('stopwords-ko.txt', encoding='utf-8') as f:
        for word in f:
            sw.add(word.replace('\n', ''))
    tokens = []
    from eunjeon import Mecab
    mecab = Mecab()
    for t in mecab.morphs(txt):
        if t not in sw and len(t) > 1:
            tokens.append(t)    
    return tokens

# nsmc 텍스트 전처리 함수 
def make(data):
    result = []
    for line in data:
        v = preprocessing(line[0])
        if v:
            result.append((v, line[1])) # 전처리한 텍스트와 label을 튜플로 추가
    return result

In [6]:
train_data = make(train)

In [7]:
train_data[:3]

[(['진짜', '짜증', '네요', '목소리'], '0'),
 (['포스터', '보고', '초딩', '영화', '오버', '연기', '가볍', '구나'], '1'),
 (['너무', '밓었', '추천', '한다'], '0')]

In [8]:
test_data = make(test)

In [9]:
test_data[:3]

[(['평점', '나쁘', '짜리', '더더욱', '잖아'], '0'),
 (['지루', '은데', '완전', '막장'], '0'),
 (['어도', '텐데', '나와서', '심기', '불편'], '0')]

In [10]:
import pickle
with open('train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f)
with open('test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)

In [14]:
len(train_data), len(test_data)

(146124, 48760)

In [15]:
# 피클로 불러오기
import pickle
with open('train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

### NSMC 분류

In [18]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import numpy as np

In [24]:
# 데이터 전처리
# x = ['', '', '', '', '',...]
# y = ['0', '1', '0',...]
train_x = [' '.join(d) for d, _ in train_data]
test_x = [' '.join(d) for d, _ in test_data]
train_y = [ l for _, l in train_data]
test_y = [ l for _, l in test_data]

In [27]:
train_x[:3], train_y[:3]

(['진짜 짜증 네요 목소리', '포스터 보고 초딩 영화 오버 연기 가볍 구나', '너무 밓었 추천 한다'], ['0', '1', '0'])

In [26]:
pip = Pipeline([
    ('vect', CountVectorizer()),
    ('model', SVC())
])

In [28]:
# 학습
pip.fit(train_x, train_y)
pred = pip.predict(test_x)

In [30]:
# 평가
(pred == test_y).mean()

0.8145406070549631

In [32]:
from eunjeon import Mecab
mecab = Mecab()
pip = Pipeline([
    ('vect', CountVectorizer(analyzer=mecab.pos)),
    ('model', SVC())
])
pip.fit(train_x[:1000], train_y[:1000])  # 1000개만 학습시켜봄
pred_ = pip.predict(test_x)

In [33]:
(pred_ == test_y).mean()

0.6885767022149303

In [44]:
# 실제 텍스트 데이터로 바로 예측
pip.predict(['넌 참 착한 아이구나', '아주 재밌는 영화네요', '재밌는 영화네요', '재미없네요', '다시 보고싶어요'])

array(['0', '1', '1', '0', '1'], dtype='<U1')

In [45]:
from eunjeon import Mecab
mecab = Mecab()
pip_ = Pipeline([
    ('vect', CountVectorizer(analyzer=mecab.pos)),
    ('model', SVC())
])
pip_.fit(train_x, train_y)  # 1000개만 학습시켜봄
pred_ = pip_.predict(test_x)

In [46]:
(pred_ == test_y).mean()

0.8109105824446268

In [47]:
pip_.predict(['넌 참 착한 아이구나', '아주 재밌는 영화네요', '재밌는 영화네요', '재미없네요', '다시 보고싶어요'])

array(['0', '1', '1', '0', '1'], dtype='<U1')